# 通過データのRSSI値分析
通過データをフォルダ単位で実施し、CSV,EPC毎にRSSI値の基本統計量を算出する。
- 実施日：2018/02/08 15:45  
- 使用データ：[20180124_ant_naname_24dBm](http://http://13.113.187.194:1337/csv/lists?dir=20180208_archTag_5sozai)

----

### ライブラリ/設定をロード

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider
import math
import json
import os.path
import urllib.parse
import urllib.request
from IPython.core.display import display
import re

import LibImporter
import lib.TestCsv as TestCsv
import lib.judge_logic_20180208 as judge_logic

%matplotlib inline
pd.options.display.max_rows=5 # 最大表示数

### 通過データのダウンロード及び整形

In [2]:
directory = '20180208_archTag_5sozai' # 検証対象のフォルダ名を指定
csvs = TestCsv.download_dir(directory) # 指定したフォルダーをダウンロードして整形
csvs # 結果の確認

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\taichi.ariyoshi\\pythoncsv/download/cache/20180208_archTag_5sozai.csv'

### 統計量の算出  
そこそこ時間掛かるので注意！  
特定のEPCについてだけ実施したい場合は、そのEPCを事前に指定すること。

---
下記の何れかを設定すれば、そのEPC(またはEPCとCSV)に限定して動作する為、高速。
- **target_epcs** : 基本統計量を求めたいEPCを指定したい場合  
例)
```
target_epcs = ['0x000000000000000000004001', '0x000000000000000000004002']
```
- target_csv_epcs : 基本統計量を求めたいEPCをCSV単位で指定したい場合  
例)
```
target_csv_epcs = {
    '20180124111756656-1-1.10965_3331-4198-4501.csv': ['0x000000000000000000004001', '0x000000000000000000004002'],
    '20180124111756656-1-1.10965_3331.csv': ['0x000000000000000000004003', '0x000000000000000000004004']
}
```

In [ ]:
# target_epcs = ['epc値1', 'epc値2']
target_epcs = ['0x000000000000000000007111', '0x000000000000000000007121', '0x000000000000000000007131', '0x000000000000000000007141'
, '0x000000000000000000007151'] # RSSIの統計量を算出したいEPCを指定する

# target_csv_epcs = {
#     'csv名1': ['epc値1', 'epc値2'],
#     'csv名2': ['epc値1', 'epc値2']
# }
target_csv_epcs = {}

In [ ]:
result = pd.DataFrame() # 空のデータフレームを作る
for name, csv in csvs.groupby(['csv', 'epc']): # CSV, EPC毎にグルーピングして、1グループずつループ
    temp = None
    if len(target_epcs) == 0 and len(target_csv_epcs) == 0:
        temp = csv[['rssi']].describe() # RSSIの統計量を算出(全て)
    elif len(target_epcs) > 0 and len(target_csv_epcs) == 0:
        temp = csv.query('epc in @target_epcs')[['rssi']].describe() # RSSIの統計量を算出(EPC指定)
    else:
        taret_epcs = target_csv_epcs[name[0]]
        temp = csv.query('epc in @taret_epcs')[['rssi']].describe() # RSSIの統計量を算出(CSV単位でEPC指定)
    temp = csv[['rssi']].describe() # RSSIの統計量を算出
    temp['csv'], temp['epc'] = name[0], name[1] # CSVとEPCのカラムを追加
    temp = temp.reset_index().set_index(['csv', 'epc']) # IndexをCSV, EPCに変更
    temp = temp.rename(columns={'index':'type'}) # 統計量のカラム名を"type"に修正
    result = result.append(temp) # 最終結果用のデータフレームに付加する
result # 結果の確認

### CSVファイルとして吐き出す

In [ ]:
# Excelで読み易い様にShift-Jisにしてる
result.to_csv(path_or_buf=f"./通過データ_{directory}_shiftjis.csv", encoding='shift-jis') # このファイルと同じディレクトリに出力